## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_file import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.49,65,99,4.54,overcast clouds
1,1,Norden,DE,53.6000,7.2000,40.98,78,100,24.40,light rain
2,2,Bulawayo,ZW,-20.1500,28.5833,63.37,88,34,6.91,scattered clouds
3,3,San Cristobal,VE,7.7669,-72.2250,78.98,46,84,3.74,broken clouds
4,4,Cape Town,ZA,-33.9258,18.4232,77.11,78,0,1.99,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.49,65,99,4.54,overcast clouds
3,3,San Cristobal,VE,7.7669,-72.2250,78.98,46,84,3.74,broken clouds
4,4,Cape Town,ZA,-33.9258,18.4232,77.11,78,0,1.99,clear sky
7,7,Bengkulu,ID,-3.8004,102.2655,75.65,86,99,3.20,overcast clouds
8,8,Ixtapa,MX,20.7000,-105.2000,78.03,53,0,8.05,clear sky
18,18,Kapaa,US,22.0752,-159.3190,77.11,70,0,1.01,clear sky
20,20,Vaini,TO,-21.2000,-175.2000,79.81,75,100,14.41,light rain
23,23,Bambous Virieux,MU,-20.3428,57.7575,80.85,74,75,18.41,light rain
24,24,Georgetown,MY,5.4112,100.3354,80.89,84,20,1.99,few clouds
26,26,Hithadhoo,MV,-0.6000,73.0833,82.40,72,32,9.35,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#No empty rows or null values | every column has the same number of values as checked by preferred_cities_df.count()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.49,overcast clouds,-23.1203,-134.9692,
3,San Cristobal,VE,78.98,broken clouds,7.7669,-72.2250,
4,Cape Town,ZA,77.11,clear sky,-33.9258,18.4232,
7,Bengkulu,ID,75.65,overcast clouds,-3.8004,102.2655,
8,Ixtapa,MX,78.03,clear sky,20.7000,-105.2000,
18,Kapaa,US,77.11,clear sky,22.0752,-159.3190,
20,Vaini,TO,79.81,light rain,-21.2000,-175.2000,
23,Bambous Virieux,MU,80.85,light rain,-20.3428,57.7575,
24,Georgetown,MY,80.89,few clouds,5.4112,100.3354,
26,Hithadhoo,MV,82.40,scattered clouds,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
#No empty rows or null values | every column has the same number of values as checked by hotels_df.count()
#print(hotel_df.loc[index, "Hotel Name"] == "")
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.49,overcast clouds,-23.1203,-134.9692,People ThankYou
3,San Cristobal,VE,78.98,broken clouds,7.7669,-72.2250,Posada Villaven C.A.
4,Cape Town,ZA,77.11,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
7,Bengkulu,ID,75.65,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
8,Ixtapa,MX,78.03,clear sky,20.7000,-105.2000,Hotel Plaza Providencia
18,Kapaa,US,77.11,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
20,Vaini,TO,79.81,light rain,-21.2000,-175.2000,Keleti Beach Resort
23,Bambous Virieux,MU,80.85,light rain,-20.3428,57.7575,Casa Tia Villa
24,Georgetown,MY,80.89,few clouds,5.4112,100.3354,Cititel Penang
26,Hithadhoo,MV,82.40,scattered clouds,-0.6000,73.0833,Scoop Guest House


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))